In [1]:
!pip install bertopic
#!pip install top2vec
!pip install octis
#!pip install contextualized-topic-models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 16.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 22.9 MB/s eta 0:00:00
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 8.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (7

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#from google.colab import drive
#drive.mount('/content/drive')
!pwd
%cd /content/drive/My Drive/PhD/FirstPaper/Final
!pwd

/content
/content/drive/My Drive/PhD/FirstPaper/Final
/content/drive/My Drive/PhD/FirstPaper/Final


In [3]:
import re
import nltk
import string
import pandas as pd

from typing import List, Tuple, Union
from octis.dataset.dataset import Dataset
from octis.preprocessing.preprocessing import Preprocessing

nltk.download("punkt")
#data preprocessing and labelling
def binary_labelling(row):
  if row['UrgentLevel'] == 4 or row['UrgentLevel'] == 5:
    return '1'
  return '0'

def CyberDS22_Zero_day_labelling(row):
  if row['UrgentLevel'] == 4 and row['AttackType'] == 'zda':
    return '1'
  elif row['UrgentLevel'] == 5 :
    return '1'
  elif row['UrgentLevel'] == 3 :
    return '1'
  return '0'

def CyberDS18_Zero_day_labelling(row):
  if row['UrgentLevel'] == 4 and row['type'] in [ '["0day"]' ]:
    return '1'
  elif row['UrgentLevel'] == 5 and row['type'] in [ '["0day"]' ]:
    return '1'
  elif row['UrgentLevel'] == 3 and row['type'] in '["vulnerability"]':
    return '1'
  return '0'

def Labelling(Cyberattack_dataset: str):
  if Cyberattack_dataset == 'Twitter_DS':
    colsName = ["User", "Username", "UTC_Date","Tweet","Binders","Permalink","Retweet_count","Likes_count", "Tweet_value","label","label2","label3","label4","UrgentLevel","AttackType"]
  else:
    colsName = ["annotation", "date", "id","relevant","text","type","UrgentLevel"]
  Twitter_DS=pd.read_csv(Cyberattack_dataset + '.csv',sep=',',usecols=colsName,parse_dates=True)
  Twitter_DS['UrgentLevel_attack'] = Twitter_DS.apply(lambda row: binary_labelling(row), axis=1)
  #Twitter_DS["UrgentLevel_attack"].value_counts()
  if Cyberattack_dataset == 'Twitter_DS':
    Twitter_DS['UrgentLevel_zeroday'] = Twitter_DS.apply(lambda row: CyberDS22_Zero_day_labelling(row), axis=1)
  else:
    Twitter_DS['UrgentLevel_zeroday'] = Twitter_DS.apply(lambda row: CyberDS18_Zero_day_labelling(row), axis=1)
    #Twitter_DS["UrgentLevel_zeroday"].value_counts()
  with open(Cyberattack_dataset +'_labelsCyberAttack.txt', 'a') as f:
    dfAsString = '\n'.join(Twitter_DS['UrgentLevel_attack'].astype(str).values)
    f.write(dfAsString)
  with open(Cyberattack_dataset +'_labelszeroday.txt', 'a') as f:
    dfAsString = '\n'.join(Twitter_DS['UrgentLevel_zeroday'].astype(str).values)
    f.write(dfAsString)

def DS_Preprocessing(Cyberattack_dataset_str: str):
  TweetsCloumn=""
  if Cyberattack_dataset_str == "Twitter_DS":
    TweetsCloumn= "Tweet"
    colsName = ["User", "Username", "UTC_Date","Tweet","Binders","Permalink","Retweet_count","Likes_count", "Tweet_value","label","label2","label3","label4","UrgentLevel","AttackType"]
  else:
    TweetsCloumn= "text"
    colsName = ["annotation", "date", "id","relevant","text","type","UrgentLevel"]
  Cyberattack_dataset = pd.read_csv(Cyberattack_dataset_str+ ".csv",sep=',',usecols=colsName)
  Cyberattack_dataset[TweetsCloumn]=Cyberattack_dataset[TweetsCloumn].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ',regex=True)
  Cyberattack_dataset[TweetsCloumn]=Cyberattack_dataset[TweetsCloumn].str.replace("[#@/:%.,_-]", " ",regex=True, flags=re.UNICODE)
  Cyberattack_dataset[TweetsCloumn]=Cyberattack_dataset[TweetsCloumn].str.replace(r"'", "", flags=re.UNICODE)
  Cyberattack_dataset[TweetsCloumn]=Cyberattack_dataset[TweetsCloumn].str.replace(r"|", " ", flags=re.UNICODE)
  Cyberattack_dataset[TweetsCloumn]=Cyberattack_dataset[TweetsCloumn].str.replace(r"(", " ", flags=re.UNICODE)
  Cyberattack_dataset[TweetsCloumn]=Cyberattack_dataset[TweetsCloumn].str.replace(r")", " ", flags=re.UNICODE)
  Cyberattack_dataset[TweetsCloumn] = Cyberattack_dataset[TweetsCloumn].str.encode('ascii', 'ignore').str.decode('ascii')
  Cyberattack_dataset = Cyberattack_dataset.replace(r'\n',' ', regex=True)
  Cyberattack_dataset[TweetsCloumn] = Cyberattack_dataset[TweetsCloumn].replace('\n','', regex=True)
  Cyberattack_dataset[TweetsCloumn] = Cyberattack_dataset[TweetsCloumn].str.lower()
  Cyberattack_dataset = Cyberattack_dataset.loc[ (Cyberattack_dataset[TweetsCloumn].values != ""), :]
  Cyberattack_dataset_docs = Cyberattack_dataset[TweetsCloumn].to_list()
  Cyberattack_dataset_docs = [doc.lower().replace("\n", " ") for doc in Cyberattack_dataset_docs if len(doc) > 2]
  """Save the documents"""
  with open(str(Cyberattack_dataset_str)+".txt", mode="wt", encoding="utf-8") as ds_txtfile:
      ds_txtfile.write("\n".join(Cyberattack_dataset_docs))
  Labelling(Cyberattack_dataset_str)



def GetOctisDS(Cyberattack_dataset_str: str,labellingType: str):
  data = Dataset()
  DS_Preprocessing(Cyberattack_dataset_str)
  #data.load_custom_dataset_from_folder(str(Cyberattack_dataset_str))
  preprocessor = Preprocessing(lowercase=False,remove_punctuation=False,punctuation=string.punctuation,remove_numbers=False,lemmatize=False,language="english",
                            split=True, verbose=True,save_original_indexes=True,remove_stopwords_spacy=False)
  documents_path = Cyberattack_dataset_str+ '.txt'
  label_path= Cyberattack_dataset_str+'_'+labellingType+ '.txt'
  ds= preprocessor.preprocess_dataset(documents_path=documents_path, labels_path=label_path)
  ds.save(Cyberattack_dataset_str)
  return ds

Twitter_DS= GetOctisDS('Twitter_DS', 'labelsCyberAttack')
Twitter_DS_0day=GetOctisDS('Twitter_DS', 'labelszeroday')
Cybertweets18_ds= GetOctisDS('TopicCybertweets', 'labelsCyberAttack')
Cybertweets18_ds_0day=GetOctisDS('TopicCybertweets', 'labelszeroday')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
100%|██████████| 19232/19232 [00:00<00:00, 488715.00it/s]


created vocab
21811
words filtering done


100%|██████████| 19232/19232 [00:00<00:00, 493985.42it/s]


created vocab
21811
words filtering done


100%|██████████| 21371/21371 [00:00<00:00, 556827.81it/s]


created vocab
24895
words filtering done


100%|██████████| 21371/21371 [00:00<00:00, 552567.97it/s]


created vocab
24895
words filtering done


In [4]:
%%capture

from sentence_transformers import SentenceTransformer

# Prepare data
#Twitter_DS
train_corpus22, test_corpus22 = Twitter_DS.get_partitioned_corpus(use_validation=False)
train_corpus22 = [" ".join(words) for words in train_corpus22]
test_corpus22 = [" ".join(words) for words in test_corpus22]
#TopicCybertweets
train_corpus18, test_corpus18 = Cybertweets18_ds.get_partitioned_corpus(use_validation=False)
train_corpus18 = [" ".join(words) for words in train_corpus18]
test_corpus18 = [" ".join(words) for words in test_corpus18]
#data = [" ".join(words) for words in data.get_corpus()]

# Extract embeddings
model = SentenceTransformer("all-mpnet-base-v2")
train_embeddings22 = model.encode(train_corpus22, show_progress_bar=True)
test_embeddings22 = model.encode(test_corpus22, show_progress_bar=True)
train_embeddings18 = model.encode(train_corpus18, show_progress_bar=True)
test_embeddings18 = model.encode(test_corpus18, show_progress_bar=True)

In [5]:
import nltk
nltk.download('stopwords')
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix
from sklearn.pipeline import Pipeline
import numpy as np
import copy
import json
from numbers import Integral
from spacy.lang.en import English
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

with open('CyberAttackKeywords.txt') as f:
  cyber_words = dict((x.strip(),-5) for x in f)
SIA = SentimentIntensityAnalyzer()
SIA.lexicon.update(cyber_words)


class UrgentFocus_CountVectorizer(CountVectorizer):
    def __init__(self, input='content', encoding='utf-8', decode_error='strict',
                 strip_accents=None, lowercase=False, preprocessor=None, tokenizer=None, stop_words=None,
                 token_pattern=r"(?u)\b\w\w+\b", ngram_range=(1, 1), analyzer='word',
                 max_df=1.0, min_df=1, max_features=None, vocabulary=None, binary=False,
                 dtype=np.int64):
     super().__init__(input=input, encoding=encoding, decode_error=decode_error, strip_accents=strip_accents,
                      lowercase=lowercase, preprocessor=preprocessor, tokenizer=tokenizer, stop_words=stop_words,
                      token_pattern=token_pattern, ngram_range=ngram_range, analyzer=analyzer, max_df=max_df,
                      min_df=min_df, max_features=max_features, vocabulary=vocabulary, binary=binary, dtype=dtype)
    def set_params(self, **parameters):
      for parameter, value in parameters.items():
        setattr(self, parameter, value)
      return self
    def get_params(self, deep=True):
      return {"input":self.input, "encoding":self.encoding, "decode_error":self.decode_error, "strip_accents":self.strip_accents,
                      "lowercase":self.lowercase, "preprocessor":self.preprocessor, "tokenizer":self.tokenizer, "stop_words":self.stop_words,
                      "token_pattern":self.token_pattern, "ngram_range":self.ngram_range, "analyzer":self.analyzer, "max_df":self.max_df,
                      "min_df":self.min_df, "max_features":self.max_features, "vocabulary":self.vocabulary, "binary":self.binary, "dtype":self.dtype}

    def transform(self, raw_documents):
      #if len(raw_documents)==13460:
      #  return adjusted_matrix_train22
      #elif len(raw_documents)==2885:
      #  return adjusted_matrix_test22
      #elif len(raw_documents)==14956:
      #  return adjusted_matrix_train18
      #elif len(raw_documents)==3205:
      #  return adjusted_matrix_test18
      X = super().transform(raw_documents)
      adjusted_matrix = X.toarray()
      for i, doc in enumerate(raw_documents):
        score = SIA.polarity_scores(doc)
        if (score['compound'] < 0): #and (count>0):
          adjusted_matrix[i] *= 2
      return sparse.csr_matrix(adjusted_matrix)

      #  return super().transform(raw_documents)

    def fit_transform(self, raw_documents, y=None):
      X = super().fit_transform(raw_documents)
      adjusted_matrix = X.toarray()
      for i, doc in enumerate(raw_documents):
        score = SIA.polarity_scores(doc)
        if (score['compound'] < 0): #and (count>0):
          adjusted_matrix[i] *= 2
      return sparse.csr_matrix(adjusted_matrix)
      #X = super().fit_transform(raw_documents)
      #  adjusted_matrix = X.toarray()
      #  for i, doc in enumerate(raw_documents):
      #    score = SIA.polarity_scores(doc)
      #    if (score['compound'] < 0):
      #      adjusted_matrix[i] *= 2
      #  return super().transform(raw_documents)

In [8]:
# adapted from https://github.com/MaartenGr/BERTopic_evaluation
from octis.models.ETM import ETM
from octis.models.LDA import LDA
from octis.models.NMF import NMF
from octis.models.CTM import CTM
from octis.dataset.dataset import Dataset
from octis.evaluation_metrics.diversity_metrics import TopicDiversity,KLDivergence,InvertedRBO
from octis.evaluation_metrics.coherence_metrics import Coherence
from octis.evaluation_metrics.classification_metrics import F1Score, PrecisionScore, RecallScore, AccuracyScore, ClassificationScore
import json
import time
import itertools
import numpy as np
import pandas as pd
import nltk
from typing import Mapping, Any, List, Tuple
from bertopic import BERTopic


def train_topic_model(TMType: str, Cyberattack_dataset_str: str, params: Mapping[str, any], labellingType:str):

  train_topic_model.params= params
        # Prepare docs
  if Cyberattack_dataset_str == "Twitter_DS":
    train_corpus= train_corpus22
    test_corpus = test_corpus22
    train_embeddings= train_embeddings22
    test_embeddings = test_embeddings22

    if labellingType == "labelsCyberAttack":
      currentDS = Twitter_DS
    else:
      currentDS = Twitter_DS_0day
  else:
    train_corpus = train_corpus18
    test_corpus = test_corpus18
    train_embeddings=train_embeddings18
    test_embeddings=test_embeddings18
    if labellingType == "labelsCyberAttack":
      currentDS = Cybertweets18_ds
    else:
      currentDS = Cybertweets18_ds_0day
  if (TMType == "BERTopic") or (TMType =="CyberTopic"):
    model = BERTopic(**params)
    start = time.time()
    topics, probs = model.fit_transform(train_corpus, train_embeddings)#train_corpus
    train_topic_model.prob = probs
    train_topic_model.topic = topics
    train_topic_model.topicss = topics
    train_topic_model.topic_word_matrix =model.c_tf_idf_.toarray()
    train_topic_model.topic_document_matrix =  np.array(probs).transpose()
    topic_word_matrix =model.c_tf_idf_.toarray()
    topic_document_matrix =  np.array(probs).transpose()
    test_topics, test_probs = model.transform(test_corpus, test_embeddings)
    test_topic_document_matrix = np.array(test_probs).transpose()
    train_corpus, test_corpus = currentDS.get_partitioned_corpus(use_validation=False)
    all_words = [word for words in train_corpus for word in words] #train_corpus self.data.get_corpus()
    bertopic_topics = [
            [
                vals[0] if vals[0] in all_words else all_words[0]
                for vals in model.get_topic(i)[:10]
            ]
            for i in range(len(set(topics)) - 1)
        ]
    train_topic_model.bertopic_topics = bertopic_topics
    train_topic_model.topic_word_matrix = topic_word_matrix
    output_tm = {"topics": bertopic_topics,
                      "topic-word-matrix":topic_word_matrix,
                      "topic-document-matrix": topic_document_matrix,
                      "test-topic-document-matrix": test_topic_document_matrix}
    end = time.time()
    computation_time = end - start
  elif TMType == "LDA":
    model = LDA(**params)
    start = time.time()
    output_tm = model.train_model(currentDS)
    end = time.time()
    computation_time = end - start
  elif TMType == "CTM":
    model = CTM(**params)
    start = time.time()
    output_tm = model.train_model(currentDS)
    end = time.time()
    computation_time = end - start
  elif TMType == "NMF":
    model = NMF(**params)
    start = time.time()
    output_tm = model.train_model(currentDS)
    end = time.time()
    computation_time = end - start

  npmi = Coherence(texts=currentDS.get_corpus(), topk=10, measure="c_npmi")
  npmiScore= npmi.score(output_tm)
  coherenceCv = Coherence(texts=currentDS.get_corpus(), topk=10, measure="c_v")
  coherenceCvScore= coherenceCv.score(output_tm)
  topic_diversity = TopicDiversity(topk=10)
  topic_diversityScore= topic_diversity.score(output_tm)

  F1_Score = F1Score(dataset= currentDS,average='weighted')
  F1_ScoreScore= F1_Score.score(output_tm)
  Precision_Score= PrecisionScore(dataset= currentDS,average='weighted')
  Precision_ScoreScore= Precision_Score.score(output_tm)
  Recall_Score = RecallScore(dataset= currentDS,average='weighted')
  Recall_ScoreScore= Recall_Score.score(output_tm)
  Accuracy_Score = AccuracyScore(dataset= currentDS,average='weighted')
  Accuracy_ScoreScore= Accuracy_Score.score(output_tm)

  divergence = KLDivergence()
  divergenceScore= divergence.score(output_tm)
  invertedRBO= InvertedRBO(topk=10)
  invertedRBOScore= invertedRBO.score(output_tm)
  colsName = ["Dataset", "Dataset_Size", "Model","Params","labellingType","NumberOfTopic",
              "Computation_Time","AddedTime","npmiScore","coherenceCvScore","topic_diversityScore","TopicQuality",
              "F1_ScoreScore","Precision_ScoreScore","Recall_ScoreScore","Accuracy_ScoreScore","divergenceScore","invertedRBOScore"]
  resultsDS=pd.read_csv('results.csv',sep=',',usecols=colsName)
  NewRow= {'Dataset': Cyberattack_dataset_str, 'Dataset_Size': len(currentDS.get_corpus()), 'Model': TMType,'Params':str(params), 'labellingType':labellingType, 'NumberOfTopic':TopNo,
             'Computation_Time':computation_time,'AddedTime':time.asctime(time.gmtime(0)),'npmiScore':float(npmiScore),
             'coherenceCvScore':float(coherenceCvScore),'topic_diversityScore':float(topic_diversityScore),'TopicQuality':float(topic_diversityScore)*float(coherenceCvScore),
             'F1_ScoreScore':float(F1_ScoreScore),'Precision_ScoreScore':float(Precision_ScoreScore),'Recall_ScoreScore':float(Recall_ScoreScore),'Accuracy_ScoreScore':float(Accuracy_ScoreScore),
             'divergenceScore':float(divergenceScore), 'invertedRBOScore':float(invertedRBOScore) }
  resultsDS.loc[len(resultsDS)] = NewRow
  #NewresultsDS = pd.concat([resultsDS,pd.DataFrame(NewRow)], ignore_index = True)
  resultsDS.to_csv('results.csv',sep=',',index=False)


#Cybersecurity X Dataset 2022

In [ ]:
##BERTopic
import torch
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import KeyBERTInspired
from bertopic.representation import ZeroShotClassification
from bertopic.dimensionality import BaseDimensionalityReduction
from sklearn.linear_model import LogisticRegression
from transformers import BertTokenizer, BertForMaskedLM
#tokenizer = BertTokenizer.from_pretrained("CyBERTBaseMLMv1_1")
#representation_model = BertForMaskedLM.from_pretrained("CyBERTBaseMLMv1_1")


for i, TopNo in enumerate([ 100, 200,300,400,500,600,700,800]):
  params = {
        "embedding_model": "all-mpnet-base-v2", #all-mpnet-base-v2
        "nr_topics": TopNo,
        "verbose": True,
#         "vectorizer_model":UrgentFocus_CountVectorizer(stop_words="english"),
#        "representation_model":representation_model,
        "calculate_probabilities": True,}
  train_topic_model(TMType="BERTopic",Cyberattack_dataset_str="Twitter_DS",params=params,labellingType="labelsCyberAttack")
  train_topic_model(TMType="BERTopic",Cyberattack_dataset_str="Twitter_DS",params=params,labellingType="labelszeroday")

In [20]:
##CyberTopic
import torch
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import KeyBERTInspired
from bertopic.representation import ZeroShotClassification
from bertopic.dimensionality import BaseDimensionalityReduction
from sklearn.linear_model import LogisticRegression
from transformers import BertTokenizer, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained("CyBERTBaseMLMv1_1")
representation_model = BertForMaskedLM.from_pretrained("CyBERTBaseMLMv1_1")


for i, TopNo in enumerate([ 100, 200,300,400,500,600,700,800]):
  params = {
        "embedding_model": "all-mpnet-base-v2", #all-mpnet-base-v2
        "nr_topics": TopNo,
        "verbose": True,
        "min_topic_size":25,
         "vectorizer_model":UrgentFocus_CountVectorizer(stop_words="english"),
        "representation_model":representation_model,
        "calculate_probabilities": True,}
  train_topic_model(TMType="CyberTopic",Cyberattack_dataset_str="TopicCybertweets",params=params,labellingType="labelsCyberAttack")
  train_topic_model(TMType="CyberTopic",Cyberattack_dataset_str="TopicCybertweets",params=params,labellingType="labelszeroday")

2024-04-14 01:11:00,537 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-04-14 01:11:11,723 - BERTopic - Dimensionality - Completed ✓
2024-04-14 01:11:11,726 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-14 01:11:20,096 - BERTopic - Cluster - Completed ✓
2024-04-14 01:11:20,097 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-04-14 01:21:33,378 - BERTopic - Representation - Completed ✓
2024-04-14 01:21:33,379 - BERTopic - Topic reduction - Reducing number of topics
2024-04-14 01:31:53,548 - BERTopic - Topic reduction - Reduced number of topics from 116 to 100
2024-04-14 01:31:55,813 - BERTopic - Dimensionality - Reducing dimensionality of input embeddings.
/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
2024-04-14 0

In [9]:
##CTM
for i, TopNo in enumerate([100, 200, 300, 400,500,600,700,800]):
  dataset, custom = "Twitter_DS", True
  params = {"num_topics": TopNo,"inference_type":"combined","model_type":"LDA"}
  train_topic_model(TMType="CTM",Cyberattack_dataset_str="Twitter_DS",params=params,labellingType="labelsCyberAttack")
  train_topic_model(TMType="CTM",Cyberattack_dataset_str="Twitter_DS",params=params,labellingType="labelszeroday")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/135 [00:00<?, ?it/s]

Batches:   0%|          | 0/29 [00:00<?, ?it/s]

Batches:   0%|          | 0/29 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/octis/evaluation_metrics/diversity_metrics.py:244: RuntimeWarning: invalid value encountered in log
  divergence = np.sum(P*np.log(P/Q))
/usr/local/lib/python3.10/dist-packages/octis/evaluation_metrics/diversity_metrics.py:244: RuntimeWarning: invalid value encountered in log
  divergence = np.sum(P*np.log(P/Q))
/usr/local/lib/python3.10/dist-packages/octis/evaluation_metrics/diversity_metrics.py:244: RuntimeWarning: invalid value encountered in log
  divergence = np.sum(P*np.log(P/Q))
/usr/local/lib/python3.10/dist-packages/octis/evaluation_metrics/diversity_metrics.py:244: RuntimeWarning: invalid value encountered in log
  divergence = np.sum(P*np.log(P/Q))
/usr/local/lib/python3.10/dist-packages/octis/evaluation_metrics/diversity_metrics.py:244: RuntimeWarning: invalid value encountered in log
  divergence = np.sum(P*np.log(P/Q))
/usr/local/lib/python3.10/dist-packages/octis/evaluation_metrics/diversity_metrics.py:244: RuntimeWarning: invalid 

In [9]:
##LDA
for i, TopNo in enumerate([100, 200, 300, 400,500,600,700,800]):
  params = {"num_topics": TopNo, "random_state": 42}
  train_topic_model(TMType="LDA",Cyberattack_dataset_str="Twitter_DS",params=params,labellingType="labelsCyberAttack")
  train_topic_model(TMType="LDA",Cyberattack_dataset_str="Twitter_DS",params=params,labellingType="labelszeroday")


/usr/local/lib/python3.10/dist-packages/gensim/models/ldamodel.py:849: RuntimeWarning: overflow encountered in exp2
  perwordbound, np.exp2(-perwordbound), len(chunk), corpus_words
/usr/local/lib/python3.10/dist-packages/gensim/models/ldamodel.py:849: RuntimeWarning: overflow encountered in exp2
  perwordbound, np.exp2(-perwordbound), len(chunk), corpus_words
/usr/local/lib/python3.10/dist-packages/gensim/models/ldamodel.py:849: RuntimeWarning: overflow encountered in exp2
  perwordbound, np.exp2(-perwordbound), len(chunk), corpus_words
/usr/local/lib/python3.10/dist-packages/gensim/models/ldamodel.py:849: RuntimeWarning: overflow encountered in exp2
  perwordbound, np.exp2(-perwordbound), len(chunk), corpus_words


#Cybersecurity X Dataset 2018

In [ ]:
##BERTopic
import torch
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import KeyBERTInspired
from bertopic.representation import ZeroShotClassification
from bertopic.dimensionality import BaseDimensionalityReduction
from sklearn.linear_model import LogisticRegression
from transformers import BertTokenizer, BertForMaskedLM
#tokenizer = BertTokenizer.from_pretrained("CyBERTBaseMLMv1_1")
#representation_model = BertForMaskedLM.from_pretrained("CyBERTBaseMLMv1_1")


for i, TopNo in enumerate([ 100, 200,300,400,500,600,700,800]):
  params = {
        "embedding_model": "all-mpnet-base-v2", #all-mpnet-base-v2
        "nr_topics": TopNo,
        "verbose": True,
#         "vectorizer_model":UrgentFocus_CountVectorizer(stop_words="english"),
#        "representation_model":representation_model,
        "calculate_probabilities": True,}
  train_topic_model(TMType="BERTopic",Cyberattack_dataset_str="TopicCybertweets",params=params,labellingType="labelsCyberAttack")
  train_topic_model(TMType="BERTopic",Cyberattack_dataset_str="TopicCybertweets",params=params,labellingType="labelszeroday")

In [ ]:
##CyberTopic
import torch
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import KeyBERTInspired
from bertopic.representation import ZeroShotClassification
from bertopic.dimensionality import BaseDimensionalityReduction
from sklearn.linear_model import LogisticRegression
from transformers import BertTokenizer, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained("CyBERTBaseMLMv1_1")
representation_model = BertForMaskedLM.from_pretrained("CyBERTBaseMLMv1_1")


for i, TopNo in enumerate([ 100, 200,300,400,500,600,700,800]):
  params = {
        "embedding_model": "all-mpnet-base-v2", #all-mpnet-base-v2
        "nr_topics": TopNo,
        "verbose": True,
         "vectorizer_model":UrgentFocus_CountVectorizer(stop_words="english"),
        "representation_model":representation_model,
        "calculate_probabilities": True,}
  train_topic_model(TMType="CyberTopic",Cyberattack_dataset_str="TopicCybertweets",params=params,labellingType="labelsCyberAttack")
  train_topic_model(TMType="CyberTopic",Cyberattack_dataset_str="TopicCybertweets",params=params,labellingType="labelszeroday")

2024-04-13 03:38:23,422 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-04-13 03:38:33,369 - BERTopic - Dimensionality - Completed ✓
2024-04-13 03:38:33,372 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-04-13 03:39:49,239 - BERTopic - Cluster - Completed ✓
2024-04-13 03:39:49,240 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-04-13 03:48:35,239 - BERTopic - Representation - Completed ✓
2024-04-13 03:48:35,240 - BERTopic - Topic reduction - Reducing number of topics
2024-04-13 03:58:41,067 - BERTopic - Topic reduction - Reduced number of topics from 342 to 100
2024-04-13 03:58:42,617 - BERTopic - Dimensionality - Reducing dimensionality of input embeddings.
/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
2024-04-13 0

In [11]:
##CTM
for i, TopNo in enumerate([100, 200, 300, 400,500,600,700,800]):
  dataset, custom = "Twitter_DS", True
  params = {"num_topics": TopNo,"model_type":"LDA","inference_type":"combined"}
  train_topic_model(TMType="CTM",Cyberattack_dataset_str="TopicCybertweets",params=params,labellingType="labelsCyberAttack")
  train_topic_model(TMType="CTM",Cyberattack_dataset_str="TopicCybertweets",params=params,labellingType="labelszeroday")

Batches:   0%|          | 0/150 [00:00<?, ?it/s]

Batches:   0%|          | 0/33 [00:00<?, ?it/s]

Batches:   0%|          | 0/33 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/octis/evaluation_metrics/diversity_metrics.py:244: RuntimeWarning: invalid value encountered in log
  divergence = np.sum(P*np.log(P/Q))
/usr/local/lib/python3.10/dist-packages/octis/evaluation_metrics/diversity_metrics.py:244: RuntimeWarning: invalid value encountered in log
  divergence = np.sum(P*np.log(P/Q))
/usr/local/lib/python3.10/dist-packages/octis/evaluation_metrics/diversity_metrics.py:244: RuntimeWarning: invalid value encountered in log
  divergence = np.sum(P*np.log(P/Q))
/usr/local/lib/python3.10/dist-packages/octis/evaluation_metrics/diversity_metrics.py:244: RuntimeWarning: invalid value encountered in log
  divergence = np.sum(P*np.log(P/Q))
/usr/local/lib/python3.10/dist-packages/octis/evaluation_metrics/diversity_metrics.py:244: RuntimeWarning: invalid value encountered in log
  divergence = np.sum(P*np.log(P/Q))
/usr/local/lib/python3.10/dist-packages/octis/evaluation_metrics/diversity_metrics.py:244: RuntimeWarning: invalid 

In [10]:
##LDA
for i, TopNo in enumerate([100, 200, 300, 400,500,600,700,800]):
  params = {"num_topics": TopNo, "random_state": 42}
  train_topic_model(TMType="LDA",Cyberattack_dataset_str="TopicCybertweets",params=params,labellingType="labelsCyberAttack")
  train_topic_model(TMType="LDA",Cyberattack_dataset_str="TopicCybertweets",params=params,labellingType="labelszeroday")

/usr/local/lib/python3.10/dist-packages/gensim/models/ldamodel.py:849: RuntimeWarning: overflow encountered in exp2
  perwordbound, np.exp2(-perwordbound), len(chunk), corpus_words
/usr/local/lib/python3.10/dist-packages/gensim/models/ldamodel.py:849: RuntimeWarning: overflow encountered in exp2
  perwordbound, np.exp2(-perwordbound), len(chunk), corpus_words
/usr/local/lib/python3.10/dist-packages/gensim/models/ldamodel.py:849: RuntimeWarning: overflow encountered in exp2
  perwordbound, np.exp2(-perwordbound), len(chunk), corpus_words
/usr/local/lib/python3.10/dist-packages/gensim/models/ldamodel.py:849: RuntimeWarning: overflow encountered in exp2
  perwordbound, np.exp2(-perwordbound), len(chunk), corpus_words
/usr/local/lib/python3.10/dist-packages/gensim/models/ldamodel.py:849: RuntimeWarning: overflow encountered in exp2
  perwordbound, np.exp2(-perwordbound), len(chunk), corpus_words
/usr/local/lib/python3.10/dist-packages/gensim/models/ldamodel.py:849: RuntimeWarning: overflow